In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

from sklearn.metrics import mean_squared_error as MSE

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

pd.set_option('display.max_columns', 100)

%matplotlib inline

def plot_importances(model, error=False):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(16,6))
    plt.title("Feature importances")
    
    if error:
        std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
        plt.barh(range(train_X.shape[1]), importances[indices], yerr=std[indices], align="center")
    else:
        plt.barh(range(train_X.shape[1]), importances[indices], align="center")
    
    plt.xticks(range(train_X.shape[1]), [train_X.columns[ix] for ix in indices], rotation='vertical')
    plt.xlim([-1, train_X.shape[1]])
    plt.show()

0.792 1st place  
0.882 top 1%  
0.898 top 5%  
0.906 top 10%  
0.911 top 20%  
1.040 top 50%  
1.239 top 75%  

just guessing 0 on my test set - 1.975  
time,shop,item,price,category,last 12 counts RF(n=50,min_leaf=5) - 1.465 (41 min, n=300,min_leaf=1 estimated 224 min)  
same + last 12 general shop and item counts - 1.476 (145 min)  
same but [1,2,3,6,12] - 1.600 (42 min)
same + month, word embeddings[2,3,5] - 1.447 (74 min)  
same but all 12 specific counts - 1.672 (88 min)  
time,shop,item,price,category,last 12 counts, month, word embeddings - 1.447 (69 min)  
everything so far - 1.499 (124 min), 1.467 (112 min)  
everything min_leaf=1 - 1.563 (147 min)  

everything extra trees(n=50,min_leaf=5) - 1.400 (68 min)  
everything gradient boosting(50, min_leaf=1) - 1.360 (11 min), 1.375 (11 min), min_leaf=5 1.362 (11 min)  
everything gradient boosting 200 - 1.432 (52 min)  
gradient 50 word embeddings[1,1,2] - 1.354 (16 min), 1.357 (15 min)  
same + year, month_length, last_revenue, last 12 category counts - 1.458 (22 min)  
same + [3,6,12] means of all counts - 1.379 (20 min)  
same but months[1,2,3,6,12] - 1.952 (15 min)  
same but meanless - 1.602 (8 min)  
same but all store item counts - 1.988 (9 min)  
same minus useless single features - 1.571 (8 min) 1.568  

same but count clipped to (0,20) - 0.849 (8 min)  
same minus price and last revenue  - 0.849 (8 min)  
same + last premean revenue, total item return% - 0.849  
undo clip(0,20) for one run - 1.319  
month fix, added predict week, shop, item - 0.842; kaggle - 1.02109  
same but RF - 0.895 (57 min); kaggle - 1.11769  
g50 unregularized mean encodings - 0.882; kaggle - 1.05080  
+mõni 0.884  

every feature with gradient(50) -                                  0.840 (0.670) (24 min) kaggle 1.09106  
every feature with gradient(early=30, 0.6col, 0.8row)(500 trees) - 0.847 (0.570) (252 min) kaggle 1.11813  
every feature with RF(50) -                                        0.905 (0.187) (87 min) kaggle 1.13444  
every feature with RF(300, 0.6col) -                               0.887 (0.180) (140 min) kaggle 1.10903  
every feature with extra trees(300, 0.6col) -                      0.873 (0.185) (79 min) kaggle 1.07838  
gradient(50) + tSNE -                                              0.840 (0.671) (29 min)  

minus cheaps with gradient(early=30, 0.6col, 0.8row)(xx trees) -   0.752 (0.676) (259 min) kaggle 1.02715  
minus cheaps with RF(300, 0.6col) -                                0.744 (0.210) (173 min) kaggle 1.03641  

even harsher with gradient(early=30, 0.6col, 0.8row)(xx trees) -   0.xxx (0.xxx) ( min) kaggle x.xxxxx  
even harsher with RF(300, 0.6col) -                                0.xxx (0.xxx) ( min) kaggle x.xxxxx  
even harsher with extra trees(300, 0.6col) -                       0.xxx (0.xxx) ( min) kaggle x.xxxxx   
even harsher with xgboost(early=30, 0.6col, 0.8row)(xx trees) -    0.xxx (0.xxx) ( min) kaggle x.xxxxx  

uued definitsioonid - cheaps(possible overreliance nii all kui ka minus cheaps sees, enamasti mingi count, esimesed 1-5, loogiliselt natuke laiendada), strongs(väga kasulikud aga mitte juhid), thirds(üle mingi thresholdi mingis joonises); definitely shit(madalad peaaegu igal pool)  
minus cheaps with big deep (kernel?) xgboost (xx trees) -          0.xxx (0.xxx) ( min) kaggle x.xxxxx  

feature selection tööriistad  
kernel  
proovida juhuslikke subsete parimate hulgast

In [2]:
description = ''

show_error_bars = False
model_class = GradientBoostingRegressor
model_params = {
    'n_estimators': 50,
    #'n_jobs': -1 # RF
    'max_depth': 8, # GBT
    'subsample': 0.8, # GBT
    'max_features': 0.8,
    'learning_rate': 0.3,
    'min_samples_split': 300
}

# remember to set LAST_TRAINSET_TIME in notebooks A and B to 
# 29 for develop, 33 for submit for correct/best results, 
# otherwise mean encodings and return% will overfit
submit = False

In [3]:
#   # - decidedly useless
# - cheap/powerful
model_features = [
    'count', # label, dropped later
    'item', # need for prediction S,S,3,
    'shop', # need for prediction S,S,S,
    'time', # need for prediction 3,S,3,3
    
    'shop_city', # S,S,S,3x trash candidate, all trash
    'shop_type', # S,S,S,3x trash candidate, all trash
    'category', # S,S,S,3k trash candidate
    'larger_category', # S,S,S,3x trash candidate, all trash
    'item_return_percent', # S,3,-,
    
    'month', # 3,-,-,3 cheap candidate, old cheap
    #'year', # S,S,S,
    #'month_length', # S,S,S,
    #'holidays_in_month', # S,S,S,
    
    #'shop_mean_count', # S,S,33,3
    'category_mean_count', # S,3,S,
    'item_mean_count', # 3,-,-,2 cheap candidate, old cheap, chosen cheap, best cheap subset
    'shop_item_mean_count', # 1,-,-, cheap candidate, old cheap, chosen cheap, best cheap subset
    'shop_category_mean_count', # S,2,-,2 cheap candidate, best cheap subset
    
    'count_trend', # 3,3,-,3
    'item_count_trend', # 2,2,-, cheap candidate
    'item_price_trend', # S,S,S,2k trash candidate
    'months_since_first_item_sale', # 2,3,-, cheap candidate
    'months_since_last_item_sale', # S3,S3,-,3
    'months_since_first_sale', # 2,2,-, cheap candidate
    'months_since_last_sale', # 3,3,-,3
    
    'item_name_vector_1', # S,S,S32, trash candidate, all trash
    'item_name_vector_2', # S,S,S3, trash candidate, all trash
    'item_name_vector_3', # S,3,33, trash candidate, all trash
    #'item_name_tSNE_vector_1', # S,S,S32,
    #'item_name_tSNE_vector_2', # S,S,S32,
    'all_data_PCA_vector_1', # S,S,1131, trash candidate
    'all_data_PCA_vector_2', # S,S,223, trash candidate
    'all_data_PCA_vector_3', # S,3,223, trash candidate, all trash
    'all_data_PCA_vector_4', # S,S,322, trash candidate, all trash
    
    #'shop_price_lag_1', # S,S,S,
    #'category_price_lag_1', # S,S,S,
    #'shop_revenue_lag_1', # S,S,
    #'category_revenue_lag_1', # S,S,S,
    'item_revenue_lag_1', # S,S,2 trash candidate
    
    'count_lag_1', # 1,-,-,1 cheap candidate, old cheap, best cheap subset
    'count_lag_2', # S,-,-,2 cheap candidate, old cheap, chosen cheap
    'count_lag_3', # S,-,-,2 cheap candidate, old cheap, chosen cheap, best cheap subset
    #'count_lag_4', # S,S,2211,
    #'count_lag_5', # S,S,3,3x
    #'count_lag_6', # S,S,3,2
    #'count_lag_7', # S,S,SS33,3
    #'count_lag_8', # S,S,32,2
    #'count_lag_9', # S,S,S,33
    #'count_lag_10', # S,S,S,
    #'count_lag_11', # S,S,S,
    #'count_lag_12', # S,S,S,3
    'count_12_mean', # 3,1,-, cheap candidate, best cheap subset
    'count_6_mean', # 2,-,-, cheap candidate, old cheap
    'count_3_mean', # 1,-,-, cheap candidate, old cheap
    
    'item_count_lag_1', # 3,-,-,21 cheap candidate, old cheap
    #'item_count_lag_2', # S,S,2213,
    #'item_count_lag_3', # S,S,3,3
    #'item_count_lag_4', # 
    #'item_count_lag_5', # 
    'item_count_lag_6', # S,S,SS3, trash candidate
    #'item_count_lag_7', # 
    #'item_count_lag_8', # 
    #'item_count_lag_9', # 
    #'item_count_lag_10', # 
    #'item_count_lag_11', # 
    'item_count_lag_12', # S,S,S,3 trash candidate, all trash
    'item_count_12_mean', # 3,3,S, cheap candidate
    'item_count_6_mean', # 3,S,2212 cheap candidate, best cheap subset
    'item_count_3_mean', # S,2,-,
    
    #'shop_count_lag_1', # S,S,S,
    #'shop_count_lag_2', # S,S,S,
    #'shop_count_lag_3', # S,S,S,
    #'shop_count_lag_4', # 
    #'shop_count_lag_5', # 
    #'shop_count_lag_6', # S,S,S,
    #'shop_count_lag_7', # 
    #'shop_count_lag_8', # 
    #'shop_count_lag_9', # 
    #'shop_count_lag_10', # 
    #'shop_count_lag_11', # 
    'shop_count_lag_12', # S,S,S,32x
    #'shop_count_12_mean', # S,S,S,
    #'shop_count_6_mean', # S,S,S,
    #'shop_count_3_mean', # S,S,S,
    
    'shop_count_per_item_lag_1', # S,S,SSS2, trash candidate
    #'shop_count_per_item_lag_2', # S,S,S,
    #'shop_count_per_item_lag_3', # S,S,S,
    #'shop_count_per_item_lag_4', # 
    #'shop_count_per_item_lag_5', # 
    #'shop_count_per_item_lag_6', # S,S,S,
    #'shop_count_per_item_lag_7', # 
    #'shop_count_per_item_lag_8', # 
    #'shop_count_per_item_lag_9', # 
    #'shop_count_per_item_lag_10', # 
    #'shop_count_per_item_lag_11', # 
    'shop_count_per_item_lag_12', # S,S,33,3
    'shop_count_per_item_12_mean', # S,S,SSS3,3X
    #'shop_count_per_item_6_mean', # S,S,S,
    #'shop_count_per_item_3_mean', # S,S,S,
    
    'category_count_per_item_lag_1', # S,S,3,
    #'category_count_per_item_lag_2', # S,S,SSS2,
    #'category_count_per_item_lag_3', # S,S,S,
    #'category_count_per_item_lag_4', # 
    #'category_count_per_item_lag_5', # 
    #'category_count_per_item_lag_6', # S,S,S,
    #'category_count_per_item_lag_7', # 
    #'category_count_per_item_lag_8', # 
    #'category_count_per_item_lag_9', # 
    #'category_count_per_item_lag_10', # 
    #'category_count_per_item_lag_11', # 
    #'category_count_per_item_lag_12', # S,S,S,
    'category_count_per_item_12_mean', # S,S,S,2x
    'category_count_per_item_6_mean', # S,S,SSS2, trash candidate, all trash
    'category_count_per_item_3_mean', # S,S,SSS2,
    
    'shop_category_count_per_item_lag_1', # S,32,-,2 cheap candidate, best cheap subset
    #'shop_category_count_per_item_lag_2', # S,S,2
    #'shop_category_count_per_item_lag_3', # S,S,S33,
    #'shop_category_count_per_item_lag_4', # 
    #'shop_category_count_per_item_lag_5', # 
    #'shop_category_count_per_item_lag_6', # S,S,S,
    #'shop_category_count_per_item_lag_7', # 
    #'shop_category_count_per_item_lag_8', # 
    #'shop_category_count_per_item_lag_9', # 
    #'shop_category_count_per_item_lag_10', # 
    #'shop_category_count_per_item_lag_11', # 
    #'shop_category_count_per_item_lag_12', # S,S,S,
    'shop_category_count_per_item_12_mean', # S,S,233,
    'shop_category_count_per_item_6_mean', # S,S,2,3 trash candidate, all trash
    'shop_category_count_per_item_3_mean', # S,3,-,2x cheap candidate, best cheap subset
    
    'item_price_lag_1', # S,S,S,3 trash candidate, all trash
    'item_price_lag_2', # S3,S,-,2 trash candidate, all trash
    'item_price_lag_3', # S,S,2,2 trash candidate, all trash
    #'item_price_lag_4', # 
    #'item_price_lag_5', # 
    'item_price_lag_6', # S,S,S,3x trash candidate, all trash
    #'item_price_lag_7', # 
    #'item_price_lag_8', # 
    #'item_price_lag_9', # 
    #'item_price_lag_10', # 
    #'item_price_lag_11', # 
    #'item_price_lag_12', # S,S,S,
    'item_price_12_mean', # 2,2,-, cheap candidate, best cheap subset
    'item_price_6_mean', # S3,3,-,3 cheap candidate
    'item_price_3_mean', # S,S,2,2 
    
    'revenue_lag_1', # S,1,-,3 cheap candidate, chosen cheap, best cheap subset
    #'revenue_lag_2', # S,S,23
    #'revenue_lag_3', # S,S,S,
    #'revenue_lag_4', # 
    #'revenue_lag_5', # 
    #'revenue_lag_6', # S,S,S,
    #'revenue_lag_7', # 
    #'revenue_lag_8', # 
    #'revenue_lag_9', # 
    #'revenue_lag_10', # 
    #'revenue_lag_11', # 
    #'revenue_lag_12', # S,S,S,
    'revenue_12_mean', # S,3,-, thrash candidate, all trash
    #'revenue_6_mean', # S,3,-,
    'revenue_3_mean' # S,2,-,1 cheap candidate
]

data = pd.read_pickle('data/features_submit.p')[model_features]
print(data.info(max_cols=1))
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4555656 entries, 0 to 4555655
Columns: 33 entries, item_count_lag_1 to time
dtypes: float16(11), float32(9), int16(7), int8(6)
memory usage: 338.9 MB
None


,item_count_lag_1,all_data_PCA_vector_4,item_price_lag_3,item_price_lag_1,shop,revenue_3_mean,shop_count_per_item_12_mean,item_name_vector_3,item_return_percent,shop_city,shop_count_per_item_lag_1,count_lag_2,item_count_lag_6,item_name_vector_2,shop_category_mean_count,months_since_last_item_sale,count,category_mean_count,item_price_lag_2,count_lag_3,all_data_PCA_vector_1,revenue_12_mean,shop_category_count_per_item_6_mean,item_name_vector_1,all_data_PCA_vector_3,item_count_lag_12,item_price_lag_6,shop_type,shop_item_mean_count,category_count_per_item_6_mean,item,larger_category,time
0,40,795.806580,209.084213,168.530563,2,0.0,0.057373,0.179077,0.000479,14,0.134644,0,30,-0.162231,0.046448,-23,0,0.169922,168.811111,0,-726.214905,63.166668,0.022171,0.096497,-29.806570,0,398.994446,0,0.095215,5.074219,30,10,12
1,18,145.899826,570.881226,499.000000,2,0.0,0.057373,0.062225,0.003515,14,0.134644,0,13,0.141602,0.043701,-23,0,0.136230,496.227783,0,78.319992,349.458344,0.016769,0.903809,-1300.056030,0,699.000000,0,0.047607,3.447266,31,10,12
2,77,683.887878,207.253983,145.983337,2,0.0,0.057373,0.705078,0.000965,14,0.134644,0,79,0.124023,0.046448,-23,1,0.169922,149.000000,0,-909.651428,0.000000,0.022171,1.804688,-97.263023,189,348.998230,0,0.523926,5.074219,32,10,12
3,38,568.018188,194.372742,196.047623,2,199.0,0.057373,0.227783,0.004856,14,0.134644,2,38,0.538086,0.043701,-23,1,0.136230,192.800003,0,-581.274414,91.333336,0.016769,2.429688,-445.097900,45,194.372742,0,0.333252,3.447266,33,10,12
4,0,635.910706,0.000000,0.000000,2,0.0,0.057373,-0.443359,0.029419,14,0.134644,0,0,-0.479492,0.041321,-23,0,0.151611,0.000000,0,-1015.439575,0.000000,0.022018,0.336670,-40.430794,0,0.000000,0,0.000000,4.207031,38,10,12


In [4]:
### Development model training

if not submit:

    # Data split

    train = data[data['time'] < 30]
    #train = train.sample(frac=0.1)
    validate = data[(data['time'] >= 30) & (data['time'] < 34)]
    #(predict = data[data['time'] == 34])

    train_X = train.drop('count', axis=1)
    train_y = train['count']
    del train
    validate_X = validate.drop('count', axis=1)
    validate_y = validate['count']
    del validate

    # Train

    start = time()

    model = model_class(**model_params)
    model.fit(train_X, train_y)

    print('({} min)'.format(round((time()-start)/60, 1)))

    # Validate

    print('train RMSE: {}'.format(np.sqrt(MSE(model.predict(train_X), train_y))))
    print('validation RMSE: {}'.format(np.sqrt(MSE(model.predict(validate_X), validate_y))))
    
    # Analysis
    
    plot_importances(model, error=show_error_bars)


In [ ]:
### Submission model training

if submit:

    # Data split

    train = data[data['time'] < 33]
    validate = data[data['time'] == 33]
    predict = data[data['time'] == 34]

    train_X = train.drop('count', axis=1)
    train_y = train['count']
    del train
    validate_X = validate.drop('count', axis=1)
    validate_y = validate['count']
    del validate
    predict_X = predict.drop('count', axis=1)

    # Train

    start = time()

    model = model_class(**model_params)
    model.fit(train_X, train_y)

    print('({} min)'.format(round((time()-start)/60, 1)))

    # Validate

    print('train RMSE: {}'.format(np.sqrt(MSE(model.predict(train_X), train_y))))
    print('validation RMSE: {}'.format(np.sqrt(MSE(model.predict(validate_X), validate_y))))
    
    # Analysis
    
    plot_importances(model, error=show_error_bars)

    # Prediction

    final_prediction = model.predict(predict_X)
    print(np.max(final_prediction), 'should be <= 20')
    print(np.min(final_prediction), 'should be >= 0')
    final_prediction = np.clip(final_prediction, 0, 20)

    prediction = predict_X[['shop', 'item']]
    prediction.columns = ['shop_id', 'item_id']
    prediction['item_cnt_month'] = final_prediction
    prediction = pd.merge(prediction, pd.read_csv('data/test.csv'), how='right')
    prediction['item_cnt_month'].fillna(0, inplace=True) # maybe something better can be done?

    prediction.sort_values(by='ID', inplace=True)
    prediction = prediction[['ID', 'item_cnt_month']]
    print(len(prediction), 'should be 214200')

    print()
    print('Features:', ', '.join(model_features))
    print('Model:', type(model))
    print('Parameters:', str(model_params))
    print('Description:', description)

    prediction.to_csv('data/submission.csv', index=False)